In [0]:
from google.colab import drive
drive.mount('/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


## **------------------5개 필드 사용 ----------------------**
'ip.src', 'ip.dst', 'eth.src', 'eth.dst', 'tcp.seq',  'label' 필드 사용

In [0]:

import pandas as pd
import numpy as np
import glob

# 데이터 한꺼번에 불러와서 1개의 dataframe에 merge
all_files = glob.glob("/gdrive/My Drive/ARP/*.csv")
file_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, low_memory=False)
    file_list.append(df)

arp_data = pd.concat(file_list, axis=0, ignore_index=True)
arp_data = arp_data.loc[:, ['ip.src', 'ip.dst', 'eth.src', 'eth.dst', 'tcp.seq', 'label']]
arp_data = arp_data.dropna()

In [0]:
#데이터 모습 확인
print(len(arp_data))
print(arp_data.columns)

189055
Index(['ip.src', 'ip.dst', 'eth.src', 'eth.dst', 'tcp.seq', 'label'], dtype='object')


In [0]:
#라벨링
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

#생성
encoder = LabelEncoder()

arp_data['ip.src'] = encoder.fit_transform(arp_data['ip.src'])
arp_data['ip.dst'] = encoder.fit_transform(arp_data['ip.dst'])
arp_data['eth.src'] = encoder.fit_transform(arp_data['eth.src'])
arp_data['eth.dst'] = encoder.fit_transform(arp_data['eth.dst'])

# Min-Max Scaling (정규화)
scaler = MinMaxScaler(feature_range=(0,1))
arp_data['tcp.seq'] = scaler.fit_transform(arp_data[['tcp.seq']])

#확인
arp_data['ip.src']
arp_data['label'] #int형

21        0
22        0
23        0
28        0
30        0
         ..
194179    0
194180    0
194181    0
194182    0
194183    0
Name: label, Length: 189055, dtype: int64

In [0]:
#이제 label과 데이터로 DF를 나눠준다
df_labels= arp_data['label']
df_packets=arp_data.drop(['label'],axis='columns')

#리스트로 되돌리기
packets= df_packets.values.tolist()
labels= df_labels.values.tolist()

#길이 확인
len(packets)
#len(labels)
len(packets[0]) #5 = 'ip.src', 'ip.dst', 'eth.src', 'eth.dst', 'tcp.seq',

5

In [0]:
#test와 train으로 나눠주기
#전체189055개 패킷의 70%는 약 132,338
train_set = packets[:132338]
test_set = packets[132338:]
train_label =labels[:132338]
test_label = labels[132338:]

train_set = np.asarray(train_set)
train_label = np.asarray(train_label)
test_set = np.asarray(test_set)
test_label = np.asarray(test_label)

#잘 나눠졌나 확인하기
print(len(train_set) + len(test_set))
print(len(test_label) + len(train_label))


189055
189055


In [0]:
#모델 생성
import numpy as np
import tensorflow as tf


model = tf.keras.Sequential([
  tf.keras.layers.Conv1D(16, (2), padding='same', activation='relu', input_shape=(5, 1)),
  tf.keras.layers.MaxPooling1D(2,padding='same'),
  tf.keras.layers.Conv1D(16, (2), padding='same', activation='relu'),
  tf.keras.layers.MaxPooling1D(2, padding='same'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(15, activation='softmax')
])
model.summary() #모델 출력

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_68 (Conv1D)           (None, 5, 16)             48        
_________________________________________________________________
max_pooling1d_68 (MaxPooling (None, 3, 16)             0         
_________________________________________________________________
conv1d_69 (Conv1D)           (None, 3, 16)             528       
_________________________________________________________________
max_pooling1d_69 (MaxPooling (None, 2, 16)             0         
_________________________________________________________________
flatten_34 (Flatten)         (None, 32)                0         
_________________________________________________________________
dense_136 (Dense)            (None, 512)               16896     
_________________________________________________________________
dense_137 (Dense)            (None, 256)             

In [0]:
#학습 및 accuracy
train_set = np.expand_dims(train_set, axis=2) #reshape
test_set = np.expand_dims(test_set, axis=2) #reshape

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_set, train_label, epochs=5,
                    validation_data=(test_set, test_label))
test_loss = model.evaluate(test_set, test_label)

Epoch 1/5
4136/4136 [==============================] - 23s 5ms/step - loss: 0.0341 - accuracy: 0.9883 - val_loss: 0.0403 - val_accuracy: 0.9979
Epoch 2/5
4136/4136 [==============================] - 22s 5ms/step - loss: 0.0113 - accuracy: 0.9978 - val_loss: 0.0439 - val_accuracy: 0.9979
Epoch 3/5
4136/4136 [==============================] - 23s 6ms/step - loss: 0.0108 - accuracy: 0.9980 - val_loss: 0.1230 - val_accuracy: 0.9493
Epoch 4/5
4136/4136 [==============================] - 22s 5ms/step - loss: 0.0074 - accuracy: 0.9986 - val_loss: 0.0563 - val_accuracy: 0.9978
Epoch 5/5
1773/1773 [==============================] - 3s 2ms/step - loss: 0.0659 - accuracy: 0.9979


## **--------------------3개 필드 사용 ----------------------**
'ip.src', 'ip.dst', 'tcp.seq',  'label' 필드 사용

In [0]:
#arp data를 다시 가져와 열을 삭제한다.
df_labels2= arp_data['label']
df_packets2=arp_data.drop(['eth.src', 'eth.dst', 'label'],axis='columns')

#확인
print(df_packets2.columns)

#리스트로 되돌리기
packets2= df_packets2.values.tolist()
labels2= df_labels2.values.tolist()

#test와 train으로 나눠주기
#전체189055개 패킷의 70%는 약 132,338
train_set2 = packets2[:132338]
test_set2 = packets2[132338:]
train_label2 =labels2[:132338]
test_label2 = labels2[132338:]

train_set2 = np.asarray(train_set2)
train_label2 = np.asarray(train_label2)
test_set2 = np.asarray(test_set2)
test_label2 = np.asarray(test_label2)

#잘 나눠졌나 확인하기
print(len(train_set2) + len(test_set2))
print(len(test_label2) + len(train_label2))

Index(['ip.src', 'ip.dst', 'tcp.seq'], dtype='object')
189055
189055


In [0]:
#모델 생성
model = tf.keras.Sequential([
  tf.keras.layers.Conv1D(16, (2),padding='same', activation='relu', input_shape=(3, 1)),
  tf.keras.layers.MaxPooling1D(2 ,padding='same'),
  tf.keras.layers.Conv1D(16, (2),padding='same', activation='relu'),
  tf.keras.layers.MaxPooling1D(2,padding='same'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(15, activation='softmax')
])
model.summary() #모델 출력


#학습 및 accuracy
train_set2 = np.expand_dims(train_set2, axis=2) #reshape
test_set2 = np.expand_dims(test_set2, axis=2) #reshape

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_set2, train_label2, epochs=5,
                    validation_data=(test_set2, test_label2))
test_loss = model.evaluate(test_set2, test_label2)

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_70 (Conv1D)           (None, 3, 16)             48        
_________________________________________________________________
max_pooling1d_70 (MaxPooling (None, 2, 16)             0         
_________________________________________________________________
conv1d_71 (Conv1D)           (None, 2, 16)             528       
_________________________________________________________________
max_pooling1d_71 (MaxPooling (None, 1, 16)             0         
_________________________________________________________________
flatten_35 (Flatten)         (None, 16)                0         
_________________________________________________________________
dense_140 (Dense)            (None, 512)               8704      
_________________________________________________________________
dense_141 (Dense)            (None, 256)             